In [1]:
# !pip install opencv-python
# !pip install tensorflow

In [2]:
import os
import cv2
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam, Nadam

In [3]:
# Function to load the dataset
def load_dataset(data_dir, image_size=(128, 128)):
    images = []
    labels = []
    label_names = os.listdir(data_dir)
    
    for label in label_names:
        label_dir = os.path.join(data_dir, label)
        if os.path.isdir(label_dir):
            for img_name in os.listdir(label_dir):
                img_path = os.path.join(label_dir, img_name)
                # Read the image
                image = cv2.imread(img_path)
                if image is not None:
                    # Resize the image to the fixed size
                    image = cv2.resize(image, image_size)
                    images.append(image)
                    labels.append(label)
    
    # Convert lists to NumPy arrays
    images = np.array(images)
    labels = np.array(labels)
    
    return images, labels

# Specify the directory where your dataset is located
data_dir = 'AugmentedAlzheimerDataset'

# Load the dataset
images, labels = load_dataset(data_dir)

In [4]:
# Normalize the images
images = images / 255.0

# Encode the labels
label_encoder = LabelEncoder()
labels = label_encoder.fit_transform(labels)
labels = to_categorical(labels)

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)

In [5]:
# Build the CNN model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.2),
    Dense(len(label_encoder.classes_), activation='softmax')
])

C:\Users\ayush\Alzheimer\env\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [6]:
# Compile the model
model.compile(optimizer=Nadam(), loss='categorical_crossentropy', metrics=['accuracy'])

In [7]:
# Train the model
history = model.fit(X_train, y_train, epochs=20, validation_data=(X_test, y_test))

Epoch 1/20
850/850 ━━━━━━━━━━━━━━━━━━━━ 105s 121ms/step - accuracy: 0.5584 - loss: 0.9570 - val_accuracy: 0.7862 - val_loss: 0.4740
Epoch 2/20
850/850 ━━━━━━━━━━━━━━━━━━━━ 99s 116ms/step - accuracy: 0.8383 - loss: 0.3901 - val_accuracy: 0.8914 - val_loss: 0.2638
Epoch 3/20
850/850 ━━━━━━━━━━━━━━━━━━━━ 100s 118ms/step - accuracy: 0.9103 - loss: 0.2172 - val_accuracy: 0.9026 - val_loss: 0.2583
Epoch 4/20
850/850 ━━━━━━━━━━━━━━━━━━━━ 99s 116ms/step - accuracy: 0.9416 - loss: 0.1412 - val_accuracy: 0.9204 - val_loss: 0.2107
Epoch 5/20
850/850 ━━━━━━━━━━━━━━━━━━━━ 98s 115ms/step - accuracy: 0.9562 - loss: 0.1084 - val_accuracy: 0.9176 - val_loss: 0.2204
Epoch 6/20
850/850 ━━━━━━━━━━━━━━━━━━━━ 100s 118ms/step - accuracy: 0.9707 - loss: 0.0833 - val_accuracy: 0.9226 - val_loss: 0.2295
Epoch 7/20
850/850 ━━━━━━━━━━━━━━━━━━━━ 99s 116ms/step - accuracy: 0.9753 - loss: 0.0686 - val_accuracy: 0.9316 - val_loss: 0.2325
Epoch 8/20
850/850 ━━━━━━━━━━━━━━━━━━━━ 98s 115ms/step - accuracy: 0.9788 - loss

In [8]:
# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test Accuracy: {accuracy * 100:.2f}%')

213/213 ━━━━━━━━━━━━━━━━━━━━ 6s 30ms/step - accuracy: 0.9238 - loss: 0.3496
Test Accuracy: 93.09%


In [9]:
# Save the model
model.save('cnn_model_2.h5')

In [10]:
from tensorflow.keras.models import load_model

def load_existing_model(model_path):
    """
    Load an existing Keras model from a specified file path.

    Parameters:
    - model_path: The file path to the saved Keras model.

    Returns:
    - model: The loaded Keras model.
    """
    model = load_model(model_path)
    return model

# Example usage:
# Load the model from a file
model_path = 'cnn_model.h5'
loaded_model = load_existing_model(model_path)

# Check if the model is loaded successfully
print(loaded_model.summary())

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                      │ (None, 126, 126, 32)        │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 63, 63, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 61, 61, 64)          │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 30, 30, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_2 (Conv2D)                    │ (None, 28, 28, 128)         │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_2 (MaxPooling2D)       │ (None, 14, 14, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 25088)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 128)                 │       3,211,392 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 4)                   │             516 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 3,305,158 (12.61 MB)

 Trainable params: 3,305,156 (12.61 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2 (12.00 B)

None
